In [12]:
pip install requests

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install future

     ---------------------------------------- 0.0/840.9 kB ? eta -:--:--
     --- ----------------------------------- 71.7/840.9 kB 2.0 MB/s eta 0:00:01
     ------ ------------------------------- 143.4/840.9 kB 2.1 MB/s eta 0:00:01
     ------------ ------------------------- 276.5/840.9 kB 2.4 MB/s eta 0:00:01
     ------------------- ------------------ 430.1/840.9 kB 2.5 MB/s eta 0:00:01
     ------------------------- ------------ 563.2/840.9 kB 2.7 MB/s eta 0:00:01
     -------------------------------- ----- 716.8/840.9 kB 2.7 MB/s eta 0:00:01
     -------------------------------------  839.7/840.9 kB 2.7 MB/s eta 0:00:01
     -------------------------------------- 840.9/840.9 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492055 sha256=aaabd1ceb18cfaa9d4a698bd1bcd3fefde24a91f9b282311e1d7655208810c08
  Stored in directory: c:\users\s


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     ---------------------------------------- 0.0/177.6 MB ? eta -:--:--
     ---------------------------------------- 0.1/177.6 MB 1.7 MB/s eta 0:01:48
     ---------------------------------------- 0.2/177.6 MB 2.5 MB/s eta 0:01:12
     ---------------------------------------- 0.6/177.6 MB 3.2 MB/s eta 0:00:56
     ---------------------------------------- 0.8/177.6 MB 3.7 MB/s eta 0:00:48
     ---------------------------------------- 1.1/177.6 MB 4.0 MB/s eta 0:00:45
     ---------------------------------------- 1.5/177.6 MB 4.3 MB/s eta 0:00:41
     ---------------------------------------- 1.9/177.6 MB 4.8 MB/s eta 0:00:37
      --------------------------------------- 2.4/177.6 MB 5.2 MB/s eta 0:00:34
      --------------------------------------- 2.8/177.6 MB 5.3 MB/s eta 0:00:33
      --------------------------------------- 3.4/177.6 MB 5.6 MB/s eta 0:00:31
      --------------------------------------- 3


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd
import utm
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML
seed = 38 # Seed for train/val split

In [22]:
# Specify data paths
train_data = 'Example_data\Output\Train_test_data\Training_Tag_GPS_locations.xlsx'
test_data = 'Example_data\Output\Train_test_data\Testing_Tag_GPS_locations.xlsx'
radio_tower_xy_path = 'H:\My Drive\Colab Notebooks\RadioTelemetry\Tower_data\RTEastNorth.xlsx'

# Variable parameters
frequencies = ['1min'] # Add more for final run


In [23]:
#Function to convert geographic to projected coordinates
def from_latlon(lat, lon):
    easting, northing, zone_num, zone_letter = utm.from_latlon(lat, lon)
    return easting, northing, zone_num, zone_letter

In [24]:
def preprocess_sim_data(sim_data, data_type, freq, tower_locs):
    # Get data
    sim_dat_filt = sim_data[sim_data['Data_type'].isin(data_type)]
    
    # make column with the datetime to nearest 'freq' value (e.g. 5min)
    sim_dat_filt = sim_dat_filt.assign(DateTime = sim_dat_filt['DateAndTime'].dt.floor(freq=freq))
  
    # group by datetime, tag, tower and antenna, compute mean power and std power, pivot to antennas
    sim_dat_filt = (
            sim_dat_filt.groupby(['DateTime', 'TowerID', 'TagID', 'Antenna', 'POINT_X', 'POINT_Y'])['Power']
            .agg(['mean', 'count', np.std])
            .reset_index()
            .pivot_table(index=['DateTime', 'TowerID', 'TagID', 'POINT_X', 'POINT_Y'], columns='Antenna', values=['mean', 'count', 'std'])
            .reset_index()
        )
    
    # Rename columns
    sim_dat_filt.columns = [f"{col[0]}{col[1]}" if col[1] != "" else col[0] for col in sim_dat_filt.columns.values]
    sim_dat_filt = sim_dat_filt.rename(columns={ 'mean1': 'ant1_mean', 'mean2': 'ant2_mean', 'mean3': 'ant3_mean', 'mean4': 'ant4_mean',
                                                  'count1': 'ant1_count', 'count2': 'ant2_count', 'count3': 'ant3_count', 'count4': 'ant4_count',
                                                  'std1': 'ant1_std', 'std2': 'ant2_std', 'std3': 'ant3_std', 'std4': 'ant4_std'})
    
    # Calculate the mean std and total count across the antennas
    sim_dat_filt['mean_std'] = sim_dat_filt[['ant1_std', 'ant2_std', 'ant3_std', 'ant4_std']].mean(axis=1)
    sim_dat_filt['total_count'] = sim_dat_filt[['ant1_count', 'ant2_count', 'ant3_count', 'ant4_count']].sum(axis=1)

    # Fill missing values with 0
    sim_dat_filt = sim_dat_filt.fillna(value=0)
     
    # Calculate easting and northing from lat long
    sim_dat_filt['easting'], sim_dat_filt['northing'], sim_dat_filt['zone_num'], sim_dat_filt['zone_letter'] = from_latlon(sim_dat_filt['POINT_Y'].values, sim_dat_filt['POINT_X'].values)

    # Create a dictionary of the coordinates of the towers
    offset_dict = tower_locs.set_index('TowerID').to_dict()
    point_x = offset_dict['POINT_X']
    point_y = offset_dict['POINT_Y']

    # Standardise the coordinates so that the tower location == 0 on both the x and y axes.
    sim_dat_filt['xOffset'] = sim_dat_filt['easting'] - sim_dat_filt['TowerID'].map(point_x).fillna(0)
    sim_dat_filt['yOffset'] = sim_dat_filt['northing'] - sim_dat_filt['TowerID'].map(point_y).fillna(0)
    
    return sim_dat_filt

In [25]:
# Get training data
train_data = pd.read_excel(train_data)
train_data['DateAndTime'] = pd.to_datetime(train_data['DateAndTime'])

# Get testing data
test_data = pd.read_excel(test_data)
test_data['DateAndTime'] = pd.to_datetime(test_data['DateAndTime'])

# Get tower locations
tower_locs = pd.read_excel(radio_tower_xy_path)

In [26]:
# Set the pre-processing parameters to be used

# Fixed parameters
data_type = ['Simulated BTFS', 'BTFS'] # Simulation or Live BTF, or could do both
dimensions = ['xOffset', 'yOffset']
predictors = ['ant1_mean', 'ant2_mean', 'ant3_mean', 'ant4_mean', 'ant1_count', 'ant2_count', 'ant3_count', 'ant4_count', 'ant1_std', 'ant2_std', 'ant3_std', 'ant4_std', 'mean_std', 'total_count']
responses = ['xOffset', 'yOffset']
scoring = 'neg_mean_absolute_error'

In [ ]:
h2o.init()

In [32]:
# Initial pass over the data using lazy predict to identify possible options
freq = '5min'

for dimension in dimensions:
    sim_data_preproc = preprocess_sim_data(train_data, data_type, freq, tower_locs)
    X_train = sim_data_preproc[predictors]
    y_train = sim_data_preproc[dimension]
    train = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))

    # print(train.head(5))

    aml = H2OAutoML(max_models=20, seed=seed)
    aml.train(x=predictors, y=dimension, training_frame=train)

    lb = aml.leaderboard
    lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)
    


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
22:55:47.769: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
23:01:10.192: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


In [33]:
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_2_20230527_230110,395.13,156128,171.997,nan,156128
DeepLearning_grid_2_AutoML_2_20230527_230110_model_1,395.457,156386,175.279,nan,156386
DeepLearning_grid_3_AutoML_2_20230527_230110_model_1,395.57,156475,177.288,nan,156475
StackedEnsemble_BestOfFamily_1_AutoML_2_20230527_230110,395.713,156589,174.446,nan,156589
DeepLearning_grid_1_AutoML_2_20230527_230110_model_1,396.972,157587,177.427,nan,157587
DeepLearning_grid_3_AutoML_2_20230527_230110_model_2,397.56,158054,183.587,nan,158054
DeepLearning_grid_2_AutoML_2_20230527_230110_model_2,398.65,158922,182.691,nan,158922
DeepLearning_grid_1_AutoML_2_20230527_230110_model_2,399.902,159921,185.406,nan,159921
GBM_grid_1_AutoML_2_20230527_230110_model_2,400.064,160051,182.736,nan,160051
GBM_1_AutoML_2_20230527_230110,401.218,160976,188.873,nan,160976
